<a href="https://colab.research.google.com/github/dejanbatanjac/tf/blob/master/Keras_and_tenosrboard_using_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use TensorBoard in Colab
## Download and unzip ngrok

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-01 07:40:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.231.75.48, 52.2.175.150, 52.72.145.109, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.231.75.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  13.7MB/s    in 1.0s    

2019-05-01 07:40:09 (13.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL
Run the next cell to start the training before open the url.

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ed76e232.ngrok.io


## Run a Keras model with TensorBoard

In [5]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 8s 133us/step - loss: 0.2702 - acc: 0.9178 - val_loss: 0.0636 - val_acc: 0.9790
Epoch 2/12
60000/60000 [==============================] - 5s 75us/step - loss: 0.0910 - acc: 0.9732 - val_loss: 0.0400 - val_acc: 0.9859
Epoch 3/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0664 - acc: 0.9801 - val_loss: 0.0373 - val_acc: 0.9866
Epoch 4/12
60000/60000 [==============================] - 5s 75us/step - loss: 0.0561 - acc: 0.9836 - val_loss: 0.0365 - val_acc: 0.9885
Epoch 5/12
60000/60000 [====

## Optionally check tensorboard and ngrok PIDs and stop them

In [0]:
!ps aux | grep tensorboard

root       155  0.3  5.1 39898084 685952 ?     Sl   08:12   0:16 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root       184  107  0.0  33960  5000 pts/0    Ss+  09:37   0:01 /bin/sh -c ps aux | grep tensorboard
root       186  0.0  0.0  38196  5612 pts/0    S+   09:37   0:00 grep tensorboard


In [0]:
!ps aux | grep ngrok

root       158  0.2  0.1  19512 13724 ?        Sl   08:12   0:11 ./ngrok http 6006
root       181  108  0.0  33964  4980 pts/0    Ss+  09:37   0:01 /bin/sh -c ps aux | grep ngrok
root       183  0.0  0.0  38200  5016 pts/0    S+   09:37   0:00 grep ngrok


In [0]:
!kill 155

## Optionally clear the tensorboard records directory

In [0]:
!rm -rf log